<a href="https://colab.research.google.com/github/duyvm/leetcode-problems/blob/main/%5BMED%5D790_Domino_and_Tromino_Tiling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 790. Domino and Tromino Tiling

https://leetcode.com/problems/domino-and-tromino-tiling/description/

You have two types of tiles: a `2 x 1` domino shape and a tromino shape. You may rotate these shapes.

![img1](https://drive.google.com/uc?id=1UkaQxsVG0kr3wdTIcEYW0QZl4pkTRwL2)

Given an integer `n`, return the number of ways to tile an `2 x n` board. Since the answer may be very large, return it `modulo 10`<sup>`9`</sup>` + 7`.

In a tiling, every square must be covered by a **tile**. **Two tilings** are **different** *if and only if there are two 4-directionally adjacent cells on the board such that exactly one of the tilings has both squares occupied by a tile*.



**Constraints:**
- `1 <= n <= 1000`


**Example 1**

![img2](https://drive.google.com/uc?id=1ZxNw7RfGE65reDewvM-EHvAOEoTWAcWz)

- Explanation: The five different ways are show above.

In [ ]:
test_case1 =  {
            "input": {
                "n": 3,
            },
            "output": 5
        }

**Example 2 and so on**

In [ ]:
test_case2 =  {
            "input": {
                "n": 1,
            },
            "output": 1
        }

test_case3 =  {
            "input": {
                "n": 5,
            },
            "output": 24
        }

In [ ]:
test_cases = [test_case1, test_case2, test_case3]

In [ ]:
def run_test_cases(solution, function_name):
    for i in range(len(test_cases)):
        run_test_case(solution, function_name, i)

def run_test_case(solution, function_name, i):
    test_case = test_cases[i]
    result = getattr(solution, function_name)(**test_case["input"])
    if result != test_case["output"]:
        print(f'Failed test case {i} with input: {test_case["input"]} and expected result: {test_case["output"]} vs actual result: {result}')

# Approach 1 - Recursion  - Time Limit Exceeded

### Observations

**We can break the problem into three subproblems**

- The number of ways to tile the board with only domino tiles.

- The number of ways to tile the board with only tromino tiles.

- The number of ways to tile the board with tromino tiles and domino tiles.


**Let analyze each subproblem**

1. The number of ways to tile the board with only domino tiles.

    - We have two types: vertical domino and horizontal domino

    - For each column from `0` to `n-1`, we can choose to place **one** vertical domino tile or **two** horizontal ones
    
        - If we place vertical one: we proceed to `i+1` column

        - If we place two horizontal ones: we proceed to `i+2` column

2. The number of ways to tile the board with only tromino tiles.

    - For each pair of tromino tiles, it take up `2 x 3` contiguous cells with `2` ways to tile

    - To tile board with on ly tromino tiles, `n mod 3` must be 0

    - The number of tiling ways is `2`<sup>`n mod 3`</sup>

3. The number of ways to tile the board with tromino tiles and domino tiles.

  - The number of tromino tiles with be even
  
  - If we already placed one tromino tile, we must place **one more** to make sure there are no un-tile cells.

  - If we already place one tromino tile, the `i+1` column must be a **tromino tile** or a **horizontal domino tile**.

  - When we placed the **2nd** tromino tile, we proceed the the `i+2` column.

  - For each pair of tromino tiles, we have `2` possible combinations of tromino. Check example 1's illustration of Tromino tiles

**Concluded**

- For each column `i` from `0` to `n-1`, we have `3` choices:

  - **Choice 1**: placing a vertical domino tile

    - If we alread place `1st` tromino tile, we **can not** place vertical domino tile
    
    - Proceeed to `i+1`

  - **Choice 2**: placing 2 horizontal domino tiles

    - Proceed to `i+2`

  - **Choice 3**: placing a tromino tile

    - If it is `1st` tromino tile: Proceed to `i+1`

    - If it is `2nd` tromino tile: `x2` the count and proceed to `i+2`

### Analysis

- Time complexity: `3`<sup>`n`</sup>



### Implementation

In [ ]:
from re import L
from typing import List, Optional
from collections import defaultdict
from collections import deque
from heapq import heapify, heappush, heappop
from queue import PriorityQueue
from collections import Counter
import math

class Solution:

    MOD = 10**9 + 7

    def numTilings(self, n: int) -> int:
        return self.startTiling(False, 0, n) % self.MOD

    def startTiling(self, is_first_tromino_placed, i, n):
        if i == n and is_first_tromino_placed:
            return 0

        if i == n:
            return 1

        count = 0

        # place vertical domino tile
        if not is_first_tromino_placed:
            count += self.startTiling(is_first_tromino_placed, i+1, n)

        # place horizontal domino tile
        if i + 1 < n:
            count += self.startTiling(is_first_tromino_placed, i+2 if not is_first_tromino_placed else i+1, n)

        # place tromino tile
        if is_first_tromino_placed and i + 1 < n:
            count += 2 * self.startTiling(False, i+2, n)
        else:
            count += self.startTiling(True, i+1, n)

        return count % self.MOD

In [ ]:
run_test_cases(Solution(), "numTilings")

In [ ]:
run_test_case(Solution(), "numTilings", -1)

# Approach 2 - Dynamic Programing + Tabulation (Community hint) - Beat 55%

Credit go to: https://leetcode.com/problems/domino-and-tromino-tiling/solutions/6715436/recursion-tabulation-math-bonus-o-1-at-end-with-images-hashmap-c-python-java/

### Observations

- The idea of this method is calculating the number of ways from `i = 1` to `i = n`. For current `i`, it is computed based on result of previous `i-1`, `i-2`...

- Let array `count_arr` with `length = n+1` stores number of ways tiling the board. `count_arr[i]` is number of ways tiling board size `2 x i`

- We have:

  - At `i = 0`: `count_arr[0] = 0` -> no board = no tiling

  - At `i = 1`: `count_arr[1] = 1` -> place only vertical domino tile

  - At `i = 2`: `count_arr[0] = 2` -> two vertical domino tiles or two horizontal domino tiles

  - At `i = 3`: `count_arr[0] = 5` -> check example 1's explanation

  - At `i >=4`, we consider following cases:

    - Initialize value
            
            count_arr[i] = 0

    - Place vertical domino tile at column `i` case:
    
            count_arr[i] += count_arr[i-1]

    - Place horizontal domino tile at column `i-1` case: the horizontal tile will span from two columns `i-1` and `i`:
        
            count_arr[i] += count_arr[i-2]

    - Place pair L-shaped tromino tiles:

      With range `>= 3`, we can place a pair of Tromino tiles. For each placing, we must double its count because we have 2 combination for each placing.

      * Note: for `count_arr[0]`, we initialize it with `1`

            count_arr[i] += 2*count_arr[i-3] + 2*count_arr[i-4] + ... + 2*count_arr[1] + 2*count_arr[0]
            　　　　　　　　+= 2* (count_arr[i-3] + count_arr[i-4] + ... + count_arr[1] + count_arr[0])

      Simplified:

            count_arr[i] += 2* (count_arr[i-3] + ... + count_arr[0])
                    
    - Sum up:
              
            count_arr[i] = count_arr[i-1] + count_arr[i-2] + 2*(count_arr[i-3] + ... + count_arr[0])
                         = count_arr[i-1] + count_arr[i-2] + count_arr[i-3] + count_arr[i-3] + 2*(count_arr[i-4] + ... + count_arr[0]) (A)

      Furthermore, we have:

            count_arr[i-1] = count_arr[i-2] + count_arr[i-3] + 2*(count_arr[i-4] + ... + count_arr[0])
            ->
            count_arr[i-2] + count_arr[i-3] = count_arr[i-1] - 2*(count_arr[i-4] + ... + count_arr[0]) (B)
    
      Replace B into A we got final fomula

             count_arr[i] = count_arr[i-1] + count_arr[i-1] - 2*(count_arr[i-4] + ... + count_arr[0]) + count_arr[i-3] + 2*(count_arr[i-4] + ... + count_arr[0])
             count_arr[i] = 2*count_arr[i-1] + count_arr[i-3]

                         

### Analysis

- Time complexity: `O(n)`

In [ ]:
from re import L
from typing import List, Optional
from collections import defaultdict
from collections import deque
from heapq import heapify, heappush, heappop
from queue import PriorityQueue
from collections import Counter
import math

class Solution:

    MOD = 10**9 + 7

    def numTilings(self, n: int) -> int:
        count_arr = [1] * (n+1)
        if n == 1:
            return count_arr[-1]

        count_arr[2] = 2

        for i in range(3, n+1):
            count_arr[i] = (2*count_arr[i-1] + count_arr[i-3]) % self.MOD

        return count_arr[-1]

In [ ]:
run_test_cases(Solution(), "numTilings")

# Approach 3 - Optimized Approach 2

### Observations

- Look at approach 2, we only need to store three value to compute `count_arr[i]`: `count_arr[i-1]`, `count_arr[i-2]`, `count_arr[i-3]`. We can just store these three values in array with `length = 3`.
                         

### Analysis

- Time complexity: `O(n)`

In [ ]:
from re import L
from typing import List, Optional
from collections import defaultdict
from collections import deque
from heapq import heapify, heappush, heappop
from queue import PriorityQueue
from collections import Counter
import math

class Solution:

    MOD = 10**9 + 7

    def numTilings(self, n: int) -> int:
        count_arr = [1] * 3
        if n == 1:
            return 1

        count_arr[2] = 2

        for i in range(3, n+1):
            current = (2*count_arr[-1] + count_arr[-3]) % self.MOD
            count_arr[-3] = count_arr[-2]
            count_arr[-2] = count_arr[-1]
            count_arr[-1] = current

        return count_arr[-1]

In [ ]:
run_test_cases(Solution(), "numTilings")